In [1]:
# RSA Encryption Algorithm: An In-Depth Review
# By Subhajit Das and Daniel Hoogasian
# Spring 2023 CS566

'''
Code in this cell adapted from:

Title: modinverse.py
Retrievel Date: 4/13/2023 
Source: https://gist.github.com/ofaurax/6103869014c246f962ab30a513fb5b49
'''

import random
import math

# Greatest common denominator function
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    g, y, x = egcd(b%a,a)
    return (g, x - (b//a) * y, y)

# Modular inverse function
def modular_inverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('No modular inverse')
    return x%m

In [2]:
'''
Code in this cell adapted from:

Title: Foundations of Computer Science
Retrievel Date: 4/13/2023 
Source: https://www.teach.cs.toronto.edu/~csc110y/fall/notes/08-cryptography/05-rsa-cryptosystem-implementation.html
'''
# Function to generate key
def rsa_generate_key(p: int, q: int) -> \
        tuple[tuple[int, int, int], tuple[int, int]]:

    # Compute n
    n = p * q

    # Compute phi(n)
    phi_n = (p - 1) * (q - 1)

    # Randomly choose number for e until condition (i.e., it is relatively prime to phi(n))
    # is satisfied
    e = random.randint(2, phi_n - 1)
    while math.gcd(e, phi_n) != 1:
        e = random.randint(2, phi_n - 1)

    # Calculate d
    d = modular_inverse(e, phi_n)
    print("Public key: ",(n, e), "\nPrivate key: ", (p, q, d))
    
    return ((p, q, d), (n, e))

# Encryption function
def rsa_encrypt_text(public_key: tuple[int, int], plaintext: str) -> str:
    n, e = public_key

    encrypted = ''
    for letter in plaintext:
        encrypted = encrypted + chr((ord(letter) ** e) % n)
    print("Encrypted message: ", encrypted)
    
    return encrypted

# Decryption function 
def rsa_decrypt_text(private_key: tuple[int, int, int], ciphertext: str) -> str:
    p, q, d = private_key
    n = p * q

    decrypted = ''
    for letter in ciphertext:
        decrypted = decrypted + chr((ord(letter) ** d) % n)
    print("Decrypted message: ", decrypted)
    
    return decrypted

In [3]:
# Generate keys with our example
rsa_chosen_numbers = rsa_generate_key(7, 19) # enter sample primes (we used 7 & 19)
rsa_encrypt = rsa_encrypt_text(rsa_chosen_numbers[1], "RSA") # enter sample text (we used RSA)
rsa_decrypt = rsa_decrypt_text(rsa_chosen_numbers[0], rsa_encrypt)

Public key:  (133, 77) 
Private key:  (7, 19, 101)
Encrypted message:  }X
Decrypted message:  RSA


In [4]:
'''
Code in this and the next cell adapted from:

Title: How to generate Large Prime numbers for RSA Algorithm
Retrievel Date: 4/23/2023 
Source: https://www.geeksforgeeks.org/how-to-generate-large-prime-numbers-for-rsa-algorithm/
'''
# Generate keys from list of pre generated primes
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]

p = random.choice(first_primes_list)
q = random.choice(first_primes_list)

rsa_chosen_numbers = rsa_generate_key(p, q)
rsa_encrypt = rsa_encrypt_text(rsa_chosen_numbers[1], "RSA") # enter sample text (we used RSA)
rsa_decrypt = rsa_decrypt_text(rsa_chosen_numbers[0], rsa_encrypt)

Public key:  (3601, 1271) 
Private key:  (277, 13, 2327)
Encrypted message:  ̰Ζ׊
Decrypted message:  RSA


In [5]:
# Generate large prime numbers (this portion functions well on a traditional machine)

# Generate random number for prime candidate
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)

# Division with First Primes (Low-Level Primality Test)
def getLowLevelPrime(n):
    while True:
        # Obtain a random number
        pc = nBitRandom(n)
 
        # Test divisibility by pre-generated
        # primes
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        else:
            return pc

# Rabin Miller Primality Test (High-Level Primality Test)        
def isMillerRabinPassed(mrc):
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Set number of trials here
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True
 
if __name__ == '__main__':
    while True:
        n = 1024
        prime_candidate = getLowLevelPrime(n)
        if not isMillerRabinPassed(prime_candidate):
            continue
        else:
            print(n, "bit prime p is: \n", prime_candidate, "\n")
            break
p = prime_candidate

if __name__ == '__main__':
    while True:
        n = 1024
        prime_candidate_2 = getLowLevelPrime(n)
        if not isMillerRabinPassed(prime_candidate_2):
            continue
        else:
            print(n, "bit prime q is: \n", prime_candidate_2)
            break
q = prime_candidate_2

1024 bit prime p is: 
 95194907999042871840851778632615840623660582875195149073637134830043472547194324680940866405181181381840831074293369448004421045321867812270400214127795816220538187564374933637493735268467867158105398789216502441093868158408176843541556924419032130854641049320717278940627191749974884190374905478100112323799 

1024 bit prime q is: 
 178138537473847906216395127367770637235525491864060087365392644609001564264368897744464436742649472101651929356044158026980204731222020283850585182860333935715408523262684300387800947252195693177362149324392480109596203757693281829106080270920120445619052929580083539501396267479100538159297487096293684027887


In [6]:
# This runs RSA with large primes (this part is too computationally expensive to run on a traditional machine)
'''
rsa_chosen_numbers = rsa_generate_key(p, q)
rsa_encrypt = rsa_encrypt_text(rsa_chosen_numbers[1], "RSA") # enter sample text (we used RSA)
rsa_decrypt = rsa_decrypt_text(rsa_chosen_numbers[0], rsa_encrypt)
'''

'\nrsa_chosen_numbers = rsa_generate_key(p, q)\nrsa_encrypt = rsa_encrypt_text(rsa_chosen_numbers[1], "RSA") # enter sample text (we used RSA)\nrsa_decrypt = rsa_decrypt_text(rsa_chosen_numbers[0], rsa_encrypt)\n'